In [1]:
import cv2
import mediapipe as mp
import math

2025-05-07 22:18:50.621620: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 22:18:50.781304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746677930.837499   50731 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746677930.852081   50731 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746677930.977882   50731 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def distancia(p1, p2):
    return math.sqrt((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2)

In [4]:
# funcion para determinar si un dedo esta levantado o no

# tip = punta
# pip = primera articulacion
# dip = segunda articulacion
# mcp = metacarpiana (base del dedo)
def dedo_levantado(landmarks, dedo_tip, dedo_dip, dedo_pip, dedo_mcp, umbral=0.1):
    # comparar la posición "y" del punto a la punta con los puntos anteriores
    return (
        landmarks[dedo_tip].y < landmarks[dedo_dip].y
        and landmarks[dedo_dip].y < landmarks[dedo_pip].y
        and landmarks[dedo_pip].y < landmarks[dedo_mcp].y
    )



In [5]:
ruta = "./public/img/jpg/manos.jpg"

In [6]:
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5,
) as hands:
    image = cv2.imread(ruta)
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    print("handedness:", results.multi_handedness)

    if results.multi_hand_landmarks:
        # verificar si cada dedo está levantado
        dedos_levantados = []

        for hand_landmarks in results.multi_hand_landmarks:
            # dibujar los puntos de la mano
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            )

            # obtener las marcas como una lista
            landmarks = hand_landmarks.landmark

            # definir los indices de los puntos de los dedos (segun MediaPipe)
            PULGAR_TIP = 4
            INDICE_TIP = 8
            MEDIO_TIP = 12
            ANULAR_TIP = 16
            MEÑIQUE_TIP = 20

            # PUNTA DE CADA DEDO, TIP, DIP, PIP, MCP
            dedos = {
                "pulgar": (PULGAR_TIP, 2, 1, 0),
                "indice": (INDICE_TIP, 6, 5, 0),
                "medio": (MEDIO_TIP, 10, 9, 0),
                "anular": (ANULAR_TIP, 14, 13, 0),
                "menique": (MEÑIQUE_TIP, 18, 17, 0),
            }

            for nombre, (tip, dip, pip, mcp) in dedos.items():
                if dedo_levantado(landmarks, tip, dip, pip, mcp):
                    dedos_levantados.append(nombre)

            print("Dedos levantados:", dedos_levantados)

            # mostrar los dedos levantados enl a imagen
            cv2.putText(
                image,
                f"Dedos: {','.join(dedos_levantados)}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
            )
            
        # mostrar la imagen
        cv2.imshow("Mano", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

I0000 00:00:1746677975.948362   50731 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1746677975.952249   50881 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.5-arch1.1), renderer: AMD Radeon 610M (radeonsi, raphael_mendocino, LLVM 19.1.7, DRM 3.61, 6.12.27-1-lts)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746677975.967272   50866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746677975.975833   50863 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746677976.046130   50857 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
qt.qpa.plugin: Could not find the Qt platform plugin "waylan

handedness: [classification {
  index: 0
  score: 0.986937106
  label: "Left"
}
, classification {
  index: 1
  score: 0.946240902
  label: "Right"
}
]
Dedos levantados: ['pulgar', 'indice', 'medio', 'anular', 'menique']
Dedos levantados: ['pulgar', 'indice', 'medio', 'anular', 'menique', 'pulgar', 'indice', 'medio', 'anular', 'menique']
